In [1]:
import pandas as pd
import re
import time
import numpy as np

In [2]:
e2u_file = '../ppi_ml/annotations/uniprot/EntrezToUniprot_clean.csv'
u2og_file = '../ppi_ml/data/og_proteomes/nog_mapping/human.euNOG.diamond.mapping.2759'
fu_file = '../human_disease/data/Fu-etal_Supplementary_Table11_tophits.csv'
leca_file = '../ppi_ml/annotations/lists/leca_nogs.txt'

In [3]:
def map_ids(x, d):
    try:
        new_id = d[x]
        return(new_id)
    except:
        print(f'Warning: {x} not mapped.')
        return(x)

In [4]:
leca_nogs = [line.strip() for line in open(leca_file, 'r')]
leca_nogs = set(leca_nogs)

In [5]:
u2og = pd.read_csv(u2og_file, sep='\t')
u2og['uniprot_id'] = u2og['ProteinID'].str.extract(r'(?<=\|)(.*)(?=\|)')
og_dict = dict()
og_issues = []
og_dupes = []
for i in range(len(u2og)):
    up = u2og['uniprot_id'][i]
    og = u2og['ID'][i]
    if up not in og_dict.keys():
        og_dict.update({up: og})
    else:
        if u2og['ID'][i] != og_dict[up]:
            og_issues.append(up)
            og_dict.pop(up)
        if up not in og_dupes:
            og_dupes.append(up)
print("# of total duplicated UniProt IDs = ", len(og_dupes))
print("# of UniProt IDs that map to multiple eggNOG IDs = ", len(set(og_issues)))

# of total duplicated UniProt IDs =  347
# of UniProt IDs that map to multiple eggNOG IDs =  347


In [6]:
e2u = pd.read_csv(e2u_file)
e2u['entrez_id'] = e2u['entrez_id'].astype(str).replace('\.0', '', regex=True)
e2u.dropna(inplace=True)
e2u.reset_index(drop=True, inplace=True)

ensembl_dict = dict()
ensembl_issues = []
ensembl_dupes = []
for i in range(len(e2u)):
    up = e2u['uniprot_sp_id'][i]
    ens = e2u['gene_stable_id'][i]
    if ens in ensembl_issues:
        continue
    elif ens not in ensembl_dict.keys():
        ensembl_dict.update({ens: up})
    else:
        if e2u['uniprot_sp_id'][i] != ensembl_dict[ens]:
            ensembl_issues.append(ens)
            ensembl_dict.pop(ens)
        if ens not in ensembl_dupes:
            ensembl_dupes.append(ens)
print("# of total Ensembl IDs uniquely mapped to UniProt IDs = ", len(ensembl_dict))
print("# of total duplicated Ensembl IDs = ", len(ensembl_dupes))
print("# of Ensembl IDs that map to multiple UniProt IDs = ", len(set(ensembl_issues)))

# of total Ensembl IDs uniquely mapped to UniProt IDs =  21236
# of total duplicated Ensembl IDs =  15456
# of Ensembl IDs that map to multiple UniProt IDs =  41


In [7]:
fu = pd.read_csv(fu_file)
fu = fu.sort_values('FDR_TADA_ASD', ascending=True)
#fu_top = fu.head(255)
fu_top = fu[fu['FDR_TADA_ASD'] <= 0.01]
print(f'# of hits extracted = {len(fu_top)}')
fu_top['uniprot_id'] = [map_ids(i, ensembl_dict) for i in fu_top['gene_id']]
fu_top['orthogroup'] = [map_ids(map_ids(str(i), ensembl_dict), og_dict) for i in fu_top['gene_id']]
fu_top['leca'] = ['True' if i in leca_nogs else 'False' for i in fu_top['orthogroup']]

# of hits extracted = 111


/tmp/ipykernel_410256/309657319.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fu_top['uniprot_id'] = [map_ids(i, ensembl_dict) for i in fu_top['gene_id']]
/tmp/ipykernel_410256/309657319.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fu_top['orthogroup'] = [map_ids(map_ids(str(i), ensembl_dict), og_dict) for i in fu_top['gene_id']]
/tmp/ipykernel_410256/309657319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [8]:
fu_top

,gene,gene_gencodeV33,gene_id,chromosome,ASD72,ASD185,DD309,DD477,NDD373,NDD664,...,l10BF_ASD_misB,l10BF_ASD_misA,l10BF_ASD_DEL,l10BF_ASD_DUP,l10BF_DD_PTV,l10BF_DD_misB,l10BF_DD_misA,uniprot_id,orthogroup,leca
4559,PTEN,PTEN,ENSG00000171862,10.0,True,True,True,True,True,True,...,14.020618,0.077350,0.132195,0.000000,20.714038,24.876885,0.240444,P60484,KOG2283,True
324,ADNP,ADNP,ENSG00000101126,20.0,True,True,True,True,True,True,...,0.000000,0.000000,0.000000,0.000000,131.840843,0.000000,0.000000,Q9H2P0,ENOG502QSYX,False
327,SHANK3,SHANK3,ENSG00000251322,22.0,True,True,True,True,True,True,...,0.000000,0.000000,19.704500,0.000000,68.466886,0.000000,0.000000,ENSG00000251322,ENSG00000251322,False
346,SCN2A,SCN2A,ENSG00000136531,2.0,True,True,True,True,True,True,...,20.847937,1.328200,0.803454,0.000000,33.457892,45.861926,7.804075,Q99250,KOG2301,True
98,CHD8,CHD8,ENSG00000100888,14.0,True,True,True,True,True,True,...,8.481258,1.248313,0.000000,0.000000,17.363638,0.000000,0.358431,Q9HCK8,KOG0384,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,NCKAP1,NCKAP1,ENSG00000061676,2.0,False,True,False,False,False,True,...,0.807108,0.000000,1.206512,0.050489,0.000000,0.000000,0.000000,Q9Y2A7,KOG1917,True
2723,UBR1,UBR1,ENSG00000159459,15.0,False,True,False,False,False,True,...,0.000000,0.410003,0.000000,0.068242,0.000000,0.000000,0.000000,Q8IWV7,KOG1140,True
511,TLE4,TLE4,ENSG00000106829,9.0,False,True,False,False,False,True,...,1.423751,0.000000,1.080390,0.000000,0.060939,0.000000,0.209429,Q04727,KOG0639,True
27,TRIP12,TRIP12,ENSG00000153827,2.0,False,True,True,True,True,True,...,0.943732,0.304459,0.000000,0.000000,14.277689,3.207240,0.000000,Q14669,Q14669,False


In [9]:
fu_top.groupby('leca').size()

leca
False    25
True     86
dtype: int64

In [10]:
fu_top_leca = fu_top[fu_top.leca == 'True']
fu_top_leca

,gene,gene_gencodeV33,gene_id,chromosome,ASD72,ASD185,DD309,DD477,NDD373,NDD664,...,l10BF_ASD_misB,l10BF_ASD_misA,l10BF_ASD_DEL,l10BF_ASD_DUP,l10BF_DD_PTV,l10BF_DD_misB,l10BF_DD_misA,uniprot_id,orthogroup,leca
4559,PTEN,PTEN,ENSG00000171862,10.0,True,True,True,True,True,True,...,14.020618,0.077350,0.132195,0.000000,20.714038,24.876885,0.240444,P60484,KOG2283,True
346,SCN2A,SCN2A,ENSG00000136531,2.0,True,True,True,True,True,True,...,20.847937,1.328200,0.803454,0.000000,33.457892,45.861926,7.804075,Q99250,KOG2301,True
98,CHD8,CHD8,ENSG00000100888,14.0,True,True,True,True,True,True,...,8.481258,1.248313,0.000000,0.000000,17.363638,0.000000,0.358431,Q9HCK8,KOG0384,True
15,SYNGAP1,SYNGAP1,ENSG00000197283,6.0,True,True,True,True,True,True,...,4.022717,0.149576,1.091557,0.000000,69.311990,2.141938,0.000000,Q96PV0,KOG3508,True
51,CHD2,CHD2,ENSG00000173575,15.0,True,True,True,True,True,True,...,5.901847,0.098726,1.630426,0.000000,23.366291,9.196857,0.000000,O14647,KOG0384,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3933,SYT1,SYT1,ENSG00000067715,12.0,False,True,True,True,True,True,...,2.622627,0.000000,0.000000,0.000000,0.321604,5.825833,3.500139,P21579,KOG1028,True
3052,CTNNA2,CTNNA2,ENSG00000066032,2.0,False,True,False,False,True,True,...,1.522264,0.065734,0.545490,0.000000,0.000000,1.015719,0.571170,P26232,KOG3681,True
8,NCKAP1,NCKAP1,ENSG00000061676,2.0,False,True,False,False,False,True,...,0.807108,0.000000,1.206512,0.050489,0.000000,0.000000,0.000000,Q9Y2A7,KOG1917,True
2723,UBR1,UBR1,ENSG00000159459,15.0,False,True,False,False,False,True,...,0.000000,0.410003,0.000000,0.068242,0.000000,0.000000,0.000000,Q8IWV7,KOG1140,True


In [11]:
fu_ogs = fu_top_leca['orthogroup'].to_list()
fu_list = ','.join(fu_ogs)

with open('../human_disease/network_propagation/data/autism_fu_fdr01_ogs.tsv', 'w') as f:
    f.write(f'1685\tAutism-Fu_FDR01\t{str(fu_list)}\n')